In [1]:
# Utilizar  NLP para analisar dados de redes sociais (tweets, postagens, comentários) e classificar sentimentos (positivo, negativo, neutro).

# Entrada: Dados extraídos de APIs como Twitter API ou arquivos locais.

# Apresentar as métricas de avaliação Acurácia, F1-Score, Precisão e Recall.

import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import nltk
nltk.download("stopwords")
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
# Caminhos dos arquivos
training_file = "twitter_training.csv"
validation_file = "twitter_validation.csv"

# Carregar os datasets
training_data = pd.read_csv(training_file, header=None)
validation_data = pd.read_csv(validation_file, header=None)

# Renomear colunas
training_data.columns = ['ID', 'Source', 'Sentiment', 'Text']
validation_data.columns = ['ID', 'Source', 'Sentiment', 'Text']

# Visualizar os primeiros dados
print("Treinamento:")
print(training_data.tail())

print("\nValidação:")
print(validation_data.head())

training_data = training_data.dropna()

import matplotlib.pyplot as plt

# Distribuição dos sentimentos no conjunto de treinamento
sentiment_counts = training_data['Sentiment'].value_counts()

plt.figure(figsize=(8, 5))
sentiment_counts.plot(kind='bar')
plt.title('Distribuição de Sentimentos no Conjunto de Treinamento')
plt.xlabel('Sentimentos')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)
plt.show()

# Função para limpar texto
def preprocess_text(text):
    port_stemmer = PorterStemmer()
    if not isinstance(text, str):  # Verificar se é string
        text = str(text)
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)    # Remove menções
    text = re.sub(r"#\w+", "", text)    # Remove hashtags
    text = re.sub(r"[^\w\s]", "", text) # Remove pontuação
    text = text.lower()                 # Converte para minúsculas
    #text = text.split()                 # Separa cada palavra
    #text = [port_stemmer.stem(word) for word in text if not word in stopwords.words('english')] # Coloca as palavras em sua forma base
    #text = ' '.join(text)               # Reune as palavras
    
    return text

# Aplicar a função de pré-processamento
training_data['Cleaned_Text'] = training_data['Text'].apply(preprocess_text)
validation_data['Cleaned_Text'] = validation_data['Text'].apply(preprocess_text)

# Visualizar texto limpo
print(training_data[['Text', 'Cleaned_Text']].head())

# Comprimento dos textos antes e depois do pré-processamento
training_data['Original_Length'] = training_data['Text'].apply(lambda x: len(str(x)))
training_data['Cleaned_Length'] = training_data['Cleaned_Text'].apply(lambda x: len(str(x)))

plt.figure(figsize=(10, 5))
plt.boxplot([training_data['Original_Length'], training_data['Cleaned_Length']], tick_labels=['Original', 'Limpo'])
plt.title('Comparação do Comprimento dos Textos')
plt.ylabel('Número de Caracteres')
plt.show()

# Transformar textos em vetores numéricos
vectorizer = CountVectorizer(stop_words="english")
tfidf_vectorizer = TfidfVectorizer()

# Ajustar o vetor no conjunto de treinamento e transformar os dados
X_train_vec = vectorizer.fit_transform(training_data['Cleaned_Text'])
X_val_vec = vectorizer.transform(validation_data['Cleaned_Text'])
X_train_tf = tfidf_vectorizer.fit_transform(training_data['Cleaned_Text'])
X_val_tf = tfidf_vectorizer.transform(validation_data['Cleaned_Text'])

# Rótulos
y_train = training_data['Sentiment']
y_val = validation_data['Sentiment']

# Criar e treinar o modelo
model = MultinomialNB()
model.fit(X_train_vec, y_train)
model_tf = MultinomialNB()
model_tf.fit(X_train_tf, y_train)

# Fazer previsões no conjunto de validação
y_pred = model.predict(X_val_vec)
y_pred_tf = model_tf.predict(X_val_tf)

# Calcular métricas de avaliação
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

accuracy_tf = accuracy_score(y_val, y_pred_tf)
precision_tf = precision_score(y_val, y_pred_tf, average='weighted')
recall_tf = recall_score(y_val, y_pred_tf, average='weighted')
f1_tf = f1_score(y_val, y_pred_tf, average='weighted')

# Exibir métricas
print(f"Acurácia: {(100*accuracy):.2f}%")
print(f"Precisão: {(100*precision):.2f}%")
print(f"Recall: {(100*recall):.2f}%")
print(f"F1-Score: {(100*f1):.2f}%\n")

print(f"Acurácia tfidf: {(100*accuracy_tf):.2f}%")
print(f"Precisão tfidf: {(100*precision_tf):.2f}%")
print(f"Recall tfidf: {(100*recall_tf):.2f}%")
print(f"F1-Score tfidf: {(100*f1_tf):.2f}%")
# Comparação de métricas
metrics = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']
values_count = [accuracy, precision, recall, f1]
values_tfidf = [accuracy_tf, precision_tf, recall_tf, f1_tf]

x = range(len(metrics))
plt.figure(figsize=(10, 5))
plt.bar(x, values_count, width=0.4, label='CountVectorizer', align='center')
plt.bar([p + 0.4 for p in x], values_tfidf, width=0.4, label='TF-IDF', align='center')
plt.xticks([p + 0.2 for p in x], metrics)
plt.title('Comparação de Métricas entre CountVectorizer e TF-IDF')
plt.xlabel('Métricas')
plt.ylabel('Valores')
plt.legend()
plt.show()

# Exibir relatório de classificação
print("Relatório de Classificação:")

print(classification_report(y_val, y_pred))

print(classification_report(y_val, y_pred_tf))


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


FileNotFoundError: [Errno 2] No such file or directory: 'twitter_training.csv'